In [ ]:
%pip install quandl
%pip install arch
%pip install PyPortfolioOpt
%pip install finquant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime
# import pandas_datareader as web
import quandl
import math
from statsmodels.stats.diagnostic import het_arch
from arch import arch_model
from pypfopt import EfficientFrontier,objective_functions
import finquant
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from copy import deepcopy

In [ ]:
!pip install pandas_datareader

In [ ]:
import pandas_datareader as web

In [ ]:
def modify_date(date):
    return date[:10]

In [ ]:
def data_cleaning(data):
    if (data.isna()).any().any():
        missing = np.where(data.isna())
        rows=missing[0]
        cols=missing[1]
        print(rows)
        for row in range(len(rows)):
            col=cols[row]
            if row == len(data)-1:
                data.iloc[row,col]=data.iloc[row-1,col]**2/data.iloc[row-2,col]
            elif row ==0:
                data.iloc[row,col]=data.iloc[row+1,col]**2/data.iloc[row+2,col]
            else:
                data.iloc[row,col]=math.sqrt(data.iloc[row-1,col]*data.iloc[row+1,col])
        print(data)

In [ ]:
def get_price_years_ago(data, date,time):
    year_ago=date-datetime.timedelta(days=int(time*365))
    price_year_ago=data[data.index<=year_ago]
    return price_year_ago

In [ ]:
def max_drawdown_absolute(returns):
    r = returns.add(1).cumprod()
    dd = r.div(r.cummax()).sub(1)
    mdd = dd.min()
    end = dd.argmin()
    start = r.iloc[:end].argmax()
    return abs(mdd), start, end

In [ ]:
def fit_garch(data,p,q):
    lag = max(p,q)
    length =res.shape[0]
    am = arch_model(res, vol='Garch', p=1, o=0, q=1, dist='Normal')
    split_date = one_year_monthly_return.index.values[lag-1]
    results = am.fit(last_obs=split_date)
    forecasts = results.forecast(horizon=1, start = length-lag, method='simulation', simulations=1000)
    sims = forecasts.variance
    std = np.sqrt(np.sum(sims)*12/lag)
    return std

In [ ]:
def post_allocation(dfMVOweight):

    dfMVOweight['equal']=1/len(dfMVOweight)
    numofstocks=len(dfMVOweight)
    allocationweight=allocation.loc[numofstocks,:]
    dfMVOweight['proportion']=''
    dfMVOweight['adjusted proportion']=''
    dfMVOweight['direct']=''
    residual=0
    for i in range(numofstocks):
        dfMVOweight.iloc[i,4]=dfMVOweight.iloc[i,1]/np.sum(dfMVOweight['PR_SR'])
        if (dfMVOweight.iloc[i,4]+residual/(numofstocks-i))>allocationweight[0]:
            dfMVOweight.iloc[i,5]=allocationweight[0]
        else:
            dfMVOweight.iloc[i,5]=(dfMVOweight.iloc[i,4]+residual/(numofstocks-i))
        residual+=dfMVOweight.iloc[i,4]-dfMVOweight.iloc[i,5]
    for i in range(int(allocationweight[1])):
        dfMVOweight.iloc[i,6]=allocationweight[0]
    for i in range(int(allocationweight[1]),int(allocationweight[1]+allocationweight[3])):
        dfMVOweight.iloc[i,6]=allocationweight[2]
    return dfMVOweight

In [ ]:
def output(weight):
    if stock_type == "S&P100":
        OEX= web.DataReader("^OEX", 'yahoo', start=start_swan_date, end=end_swan_date)["Adj Close"]
    elif stock_type == "S&P500":
        OEX= web.DataReader("SPY", 'yahoo', start=start_swan_date, end=end_swan_date)["Adj Close"]
    stock_price=web.DataReader(weight.index, 'yahoo', start=start_swan_date, end=end_swan_date)["Adj Close"]
    injected_result = pd.DataFrame(index = stock_price.index )
    S1=np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T
    portfolio_value = stock_price@S1.T
    injected_result["portfolio_value"]=portfolio_value
    S2 = 10000/float(OEX[0])
    injected_result["oex"]=S2*OEX
    return injected_result

# Step 1 - Preselection

In [ ]:
# Preselection parameters set up
method = "K_means"          # or avg or anyother method
stock_type = "S&P100"
end_swan = "Swan8"         #equals the last swan before the swan we are going to predict
start_swan="Swan4"
start = dt(2019,1,1)
end = dt(2020,2,15)
selected_stock_number = 19     #number of stock preselected
start_swan_date = dt(2020,2,15)
end_swan_date = dt(2021,3,23)
w1 = 0.17
w2 = 0.17
MA_days = 30
N = 7 # This parameter is set to help classification method like K-means

In [ ]:
#read all stocks and their swan beta


if stock_type == "S&P100":
    portfolio_beta = pd.read_excel("All_SwanBeta100.xlsx",index_col=0)
else:
    portfolio_beta = pd.read_excel("All_SwanBeta500.xlsx",index_col=0)

FileNotFoundError: [Errno 2] No such file or directory: 'All_SwanBeta100.xlsx'

In [ ]:
selected_swans_var=portfolio_beta.loc[:,start_swan:end_swan]
portfolio_beta =portfolio_beta[ selected_swans_var.T.isnull().sum()<selected_swans_var.shape[1]-1]

In [ ]:
portfolio_beta

,Swan-3,Swan-2,Swan-1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a,Swan10b
Ticker,,,,,,,,,,,,,,
CHTR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.583170,0.649030,0.797374,0.939407,1.060138,1.278190
GM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.026711,0.792127,0.190040,1.553517,2.447747,1.934870
KMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540529,1.230239,0.919927,1.592887,-1.141071,-0.541752
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.916183,0.944312,0.976498,0.875866,1.104899
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.870361,0.938163,0.979057,0.880849,1.117074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UNP,0.850695,2.127809,1.993802,1.088334,0.427848,0.418409,0.283910,1.275539,1.334344,1.354688,1.049957,1.188887,-0.200038,0.632258
USB,0.407208,0.375769,0.483245,0.176170,1.187362,1.307068,0.811824,1.672467,0.905422,0.995170,0.906766,1.495656,0.501194,0.825853
WFC,1.388900,1.261192,1.686449,0.558622,1.587780,1.233240,0.300985,1.698834,1.182443,0.995686,0.827634,1.470901,0.571633,1.078228


In [ ]:
corr=portfolio_beta.corr()
selected_corr=corr.loc["Swan1":end_swan,"Swan1":end_swan]
corr1={}
for i in range(selected_corr.shape[1]):
    for j in range(i+1,selected_corr.shape[0]):
        selected_corr.iloc[j,i]=np.nan

# New Section

In [ ]:
selected_corr

,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8
Swan1,1.0,-0.004268,0.111982,0.050073,0.001444,0.101963,0.079685,0.076826
Swan2,NaN,1.000000,0.549511,0.335485,0.410861,0.392490,0.033937,0.307858
Swan3,NaN,NaN,1.000000,0.151778,0.258652,0.435457,0.243705,0.224528
Swan4,NaN,NaN,NaN,1.000000,-0.032138,0.143125,0.147027,0.225411
Swan5,NaN,NaN,NaN,NaN,1.000000,0.444474,0.221512,0.208068
Swan6,NaN,NaN,NaN,NaN,NaN,1.000000,0.301317,0.482711
Swan7,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.185835
Swan8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000


In [ ]:
avg=selected_corr.mean(axis=1)
weight_swan = avg/sum(avg)
selected_beata=portfolio_beta.loc[:,start_swan:end_swan].fillna(0)
#portfolio_beta["corr_beta"]=selected_beata@weight_swan
#portfolio_beta=portfolio_beta.sort_values(by=["corr_beta"])

In [ ]:
portfolio_beta

,Swan-3,Swan-2,Swan-1,Swan1,Swan2,Swan3,Swan4,Swan5,Swan6,Swan7,Swan8,Swan9,Swan10a,Swan10b
Ticker,,,,,,,,,,,,,,
CHTR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.583170,0.649030,0.797374,0.939407,1.060138,1.278190
GM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.026711,0.792127,0.190040,1.553517,2.447747,1.934870
KMI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540529,1.230239,0.919927,1.592887,-1.141071,-0.541752
GOOG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.916183,0.944312,0.976498,0.875866,1.104899
GOOGL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.803565,1.167641,0.870361,0.938163,0.979057,0.880849,1.117074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UNP,0.850695,2.127809,1.993802,1.088334,0.427848,0.418409,0.283910,1.275539,1.334344,1.354688,1.049957,1.188887,-0.200038,0.632258
USB,0.407208,0.375769,0.483245,0.176170,1.187362,1.307068,0.811824,1.672467,0.905422,0.995170,0.906766,1.495656,0.501194,0.825853
WFC,1.388900,1.261192,1.686449,0.558622,1.587780,1.233240,0.300985,1.698834,1.182443,0.995686,0.827634,1.470901,0.571633,1.078228


In [ ]:
if method == "gamma":
    portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan].mean(axis=1)
    portfolio_beta["avg_sd"]=portfolio_beta.loc[:,start_swan:end_swan].mean(axis=1)+portfolio_beta.loc[:,start_swan:end_swan].std(axis=1,ddof=0)
    portfolio_beta=portfolio_beta.sort_values(by=["avg_sd"])
    preselected_stock = portfolio_beta.iloc[:selected_stock_number,:]
elif method =="average":
    portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan].mean(axis=1)
    #Select 18 stocks
    portfolio_beta=portfolio_beta.sort_values(by=["avg"])
    preselected_stock = portfolio_beta.iloc[:selected_stock_number,:]

elif method == "K_means":
    selected_beta =portfolio_beta.loc[:,start_swan:end_swan].dropna()
    Y_predic = selected_beta[end_swan].copy()
    Kmean = KMeans(n_clusters=N,random_state=1)#set the random_state to keep modeling the same every run
    Kmean.fit(selected_beta)
    ct = Kmean.cluster_centers_
    Kmean.labels_
    selected_beta["Group"]=Kmean.labels_
    res = ct.mean(axis=1)
    res [res<0]=0

    min_group = np.where(res==min(res))
    for i in range(N):
        Y_predic[selected_beta["Group"] == i] = res [i]
    selected_beta ["avg"]=Y_predic.values
    selected_beta=selected_beta.sort_values(by=["avg"])
    first_group=selected_beta[selected_beta["Group"]==selected_beta["Group"][0]]
    if first_group.shape[0] <10:
        second_group = selected_beta[selected_beta["Group"]==selected_beta["Group"][first_group.shape[0]]]
        preselected_stock = pd.concat([first_group,second_group],axis=0,ignore_index=True)
    else:
        preselected_stock = first_group

In [ ]:
preselected_stock

,Swan4,Swan5,Swan6,Swan7,Swan8,Group,avg
Ticker,,,,,,,
VZ,0.905674,0.332332,0.064048,0.717549,0.060184,5,0.431202
CL,0.099788,0.617542,-0.045657,0.784583,0.569564,5,0.431202
SBUX,0.289968,1.077849,0.586288,0.664628,-0.489110,5,0.431202
ABT,0.702188,0.474696,0.311079,1.073150,0.431921,5,0.431202
DUK,1.580781,0.771714,-0.322793,0.233701,-0.356250,5,0.431202
JNJ,0.403161,0.739963,0.360346,0.668742,0.555245,5,0.431202
LLY,0.746268,0.920577,0.280671,0.780342,0.061466,5,0.431202
EXC,0.519667,0.794675,0.263389,0.605246,-0.123438,5,0.431202
MRK,0.617451,0.756485,0.674926,0.995745,-0.019569,5,0.431202


In [ ]:
#portfolio_beta["avg"]=portfolio_beta.loc[:,start_swan:end_swan]*weight

In [ ]:
preselected_stock["PR"]=1-preselected_stock["avg"]
preselected_stock.head(selected_stock_number)

<ipython-input-27-dbdf54563bfa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preselected_stock["PR"]=1-preselected_stock["avg"]


,Swan4,Swan5,Swan6,Swan7,Swan8,Group,avg,PR
Ticker,,,,,,,,
VZ,0.905674,0.332332,0.064048,0.717549,0.060184,5,0.431202,0.568798
CL,0.099788,0.617542,-0.045657,0.784583,0.569564,5,0.431202,0.568798
SBUX,0.289968,1.077849,0.586288,0.664628,-0.489110,5,0.431202,0.568798
ABT,0.702188,0.474696,0.311079,1.073150,0.431921,5,0.431202,0.568798
DUK,1.580781,0.771714,-0.322793,0.233701,-0.356250,5,0.431202,0.568798
JNJ,0.403161,0.739963,0.360346,0.668742,0.555245,5,0.431202,0.568798
LLY,0.746268,0.920577,0.280671,0.780342,0.061466,5,0.431202,0.568798
EXC,0.519667,0.794675,0.263389,0.605246,-0.123438,5,0.431202,0.568798
MRK,0.617451,0.756485,0.674926,0.995745,-0.019569,5,0.431202,0.568798


# Step 2 - Download data and adjuste price based on Swan Beta

In [ ]:
#next line is used to input preselected stock from excel, if you are going to preselect by your self, use this blocks and note next one.
#selected_stock_list =  pd.read_excel("19_stickers.xlsx",header = None).values[:,0]
#swan_beta=pd.read_excel("19_stickers.xlsx",header = None,index_col=0).iloc[:,0]

In [ ]:
#Set the paramter to download tradeinfo from Yahoo Finance
selected_stock=preselected_stock
selected_stock_list=selected_stock.index.values
swan_beta=preselected_stock.loc[selected_stock_list,"avg"]

In [ ]:
selected_stock_list

array(['VZ', 'CL', 'SBUX', 'ABT', 'DUK', 'JNJ', 'LLY', 'EXC', 'MRK',
       'MCD', 'KO', 'PEP', 'PFE', 'MDLZ', 'PG', 'WMT', 'SO', 'NEE'],
      dtype=object)

In [ ]:
#download data from yahoo finance
import yfinance as yf

# yahooDataFrame= web.DataReader(selected_stock_list, 'yahoo', start=start, end=end)["Adj Close"]
yahooDataFrame = yf.download(list(selected_stock_list), start , end)["Adj Close"]

[*********************100%%**********************]  18 of 18 completed


In [ ]:
yahooDataFrame

Ticker,ABT,CL,DUK,EXC,JNJ,KO,LLY,MCD,MDLZ,MRK,NEE,PEP,PFE,PG,SBUX,SO,VZ,WMT
Date,,,,,,,,,,,,,,,,,,
2019-01-02,63.287045,51.905373,67.495857,26.005377,109.850525,39.828789,105.617340,154.936478,35.486111,61.502388,37.344528,94.145714,32.894062,79.410698,57.434700,34.845657,42.104748,28.436201
2019-01-03,60.300278,51.660076,67.471947,26.034882,108.104942,39.582676,102.335190,153.915634,35.352676,60.241245,37.252171,93.266968,31.973791,78.853935,54.943359,35.339806,42.255062,28.289974
2019-01-04,62.021324,52.141899,68.029770,26.329926,109.919312,40.371948,105.415085,156.890137,36.331238,62.055630,37.938232,95.179543,32.703926,80.463371,56.764984,35.634689,42.360287,28.466671
2019-01-07,62.950130,52.457275,67.742897,26.377134,109.214211,39.845764,105.985092,158.597382,36.580318,61.372192,37.883259,94.361092,32.878853,80.141502,56.764984,35.555004,42.630867,28.801785
2019-01-08,62.194332,53.044224,68.595558,26.784296,111.750885,40.295567,106.959618,158.931763,37.434341,61.827831,38.160324,95.265671,33.030975,80.437279,56.809631,36.567226,43.878525,29.002855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-10,82.087196,69.904320,80.020561,29.406712,134.263412,52.315704,136.797806,192.157516,53.537510,71.594261,61.199001,129.210556,30.128986,113.521347,79.998856,57.036362,47.684040,35.822220
2020-02-11,82.895950,68.783691,80.652534,30.033682,134.378342,52.175617,135.613266,194.428650,53.446720,71.251572,61.291328,129.583145,30.352106,111.712860,79.980591,57.161362,46.457108,35.868839
2020-02-12,82.542694,68.639091,81.425865,29.984987,133.582611,52.018005,134.917587,195.987854,53.519356,69.579979,60.865696,129.583145,30.073214,111.074036,80.949387,57.186371,46.092991,36.008713


In [ ]:
#set parameters to calculate predict market drop
market_drop_info = pd.read_excel("MarketDD.xlsx",index_col=0)
swan_to_predict=9
total_day=yahooDataFrame.shape[0]
t1=int(round(market_drop_info.loc[:end_swan,"T1"].mean(),0))
t1=int((np.array(market_drop_info.loc[:end_swan,"T1"])*np.array(weight_swan)).sum())
#t2=total_day-t1
t2=int(round(market_drop_info.loc[:end_swan,"T2"].mean(),0))
dd=market_drop_info.loc[:end_swan,"Actual DD"].mean()
t0=int(total_day-t1-t2)

In [ ]:
#calculate predict market drop
md = [] #Make market drop data list named md


j = 0 #Start from 0
inc = dd/(t1-1) #Calculate unit increasing market drop
for i in range(t1): #Append market drop data for t1
    md.append(round(j,4))
    j = j + inc

base = (t2+1)**(1/(dd*100)) #Calculate log base
l1 = [] #Make decreasing market drop list
for i in range(t2):
    l1.append(math.log((i+2),base)/100)

for i in range(t2): #Append market drop data for t2
    md.append(round((dd-l1[i]),4))

for i in range(t0):
    md.append(0)

market_drop = pd.DataFrame(md) #Change list into dataframe
#market_drop = market_drop.rename(columns={0: 'Swan Drop spy'}) #Change column name
market_drop1 = pd.read_csv("trainingMarketdrop.csv")

In [ ]:
market_drop.to_csv("market_drop.csv")

In [ ]:
market_drop

,0
0,0.0000
1,0.0037
2,0.0074
3,0.0111
4,0.0148
...,...
278,0.0000
279,0.0000
280,0.0000
281,0.0000


In [ ]:
swan_drop_pct=np.array(swan_beta)*np.array(market_drop)
yahooDataFrame=pd.DataFrame((1-swan_drop_pct)*np.array(yahooDataFrame),index=yahooDataFrame.index,columns=yahooDataFrame.columns)



In [ ]:
yahooDataFrame.shape


(283, 18)

In [ ]:
yahooDataFrame=yahooDataFrame.dropna(how="all",axis = 1)
selected_stock_list=yahooDataFrame.columns.values
selected_stock_number = yahooDataFrame.shape[1]
selected_stock_list

array(['ABT', 'CL', 'DUK', 'EXC', 'JNJ', 'KO', 'LLY', 'MCD', 'MDLZ',
       'MRK', 'NEE', 'PEP', 'PFE', 'PG', 'SBUX', 'SO', 'VZ', 'WMT'],
      dtype=object)

# Step3 - MVO

In [ ]:
#Set the paramter for MVO process
Portfolio_Date=end

In [ ]:
#data cleaning - filling missing data in yahooDataFrame
data_cleaning(yahooDataFrame)

In [ ]:
#calculated rf
bond_yield = quandl.get('FRED/DGS3MO')
bond_yield=np.log(bond_yield/100+1)
date=list(map(str,bond_yield.index.values))
date=list(map(modify_date,date))
week=pd.to_datetime(date).strftime("%W").values
week_1=np.roll(week,1)
week_difference=np.where(week!=week_1)
week_difference=np.delete(week_difference,0,None)
rf_week=bond_yield.iloc[week_difference-1]

In [ ]:
# Calculated R for 1 year and 0.5 year
today_price = yahooDataFrame.iloc[-MA_days:,:]
one_year_ago_data = get_price_years_ago(yahooDataFrame,end,1)
one_year_ago_date = one_year_ago_data.index.values[-1]
half_year_ago_data = get_price_years_ago(yahooDataFrame,end,0.5)
half_year_ago_date = half_year_ago_data.index.values[-1]
one_year_ago_price=yahooDataFrame.loc[one_year_ago_date:,:]
one_year_ago_price=yahooDataFrame.iloc[:MA_days,:]
half_year_ago_price=yahooDataFrame.loc[half_year_ago_date:,:]
half_year_ago_price=yahooDataFrame.iloc[:MA_days,:]

R_mv_one_year=np.mean(pd.DataFrame(np.array(today_price)/np.array(one_year_ago_price)-1,index=today_price.index,columns=today_price.columns), axis=0)
R_mv_half_year=np.mean(pd.DataFrame(np.array(today_price)/np.array(half_year_ago_price)-1,index=today_price.index,columns=today_price.columns), axis=0)



In [ ]:
# Calculate MA
MA_Data=yahooDataFrame.rolling(MA_days).mean().dropna(axis=0)
MA_Data=MA_Data.loc[one_year_ago_date:,:]
MA_now = MA_Data.iloc[-1,:]
MA_one_year_ago = MA_Data.iloc[0,:]
MA_half_year_ago = MA_Data.loc[half_year_ago_date,:]

In [ ]:
# Calculate daily return
daily_return = yahooDataFrame.pct_change().dropna()
log_daily_return = np.log(1+daily_return)

# Get week end stock price data
stock_date = yahooDataFrame.index.values
week=pd.to_datetime(stock_date).strftime("%W").values
week_1=np.roll(week,1)
week_difference=np.where(week!=week_1)
week_difference=np.delete(week_difference,0,None)
# Calculate weekly return

weekly_return=yahooDataFrame.iloc[week_difference-1].pct_change().dropna()
log_weekly_return = np.log(1+weekly_return)

# Get month end stock price data
stock_date = yahooDataFrame.index.values
month=pd.to_datetime(stock_date).strftime("%m").values
month_1=np.roll(month,1)
month_difference=np.where(month!=month_1)
month_difference=np.delete(month_difference,0,None)
# Calculate monthly return
monthly_return=yahooDataFrame.iloc[month_difference-1].pct_change().dropna()
log_monthly_return = np.log(1+monthly_return)

# Get semi-month end stock price data
semi_month = []
for i in range(1,len(month_difference)):
    semi_month.append((month_difference[i]-month_difference[i-1])//2+month_difference[i-1])
    semi_month.append(month_difference[i]-1)
# Calculate semi-monthly return
semi_monthly_return=yahooDataFrame.iloc[semi_month].pct_change().dropna()
log_semi_monthly_return = np.log(1+semi_monthly_return)

In [ ]:
# Calculated Max Dropdown for each stock
stock_return_one_year=daily_return.loc[one_year_ago_date:,:]
mdd=[]
for column in stock_return_one_year.columns:
    mdd .append(max_drawdown_absolute(stock_return_one_year.loc[:,column]))
mdd=pd.DataFrame(mdd,index = stock_return_one_year.columns,columns=["MAXDD","start_index","end_index"])
mar=R_mv_one_year/mdd.iloc[:,0]

In [ ]:
#calculate weekly sd for each stock
one_year_weekly_return=log_weekly_return.iloc[log_weekly_return.index >= one_year_ago_date]
weekly_std= np.std(one_year_weekly_return)*np.sqrt(one_year_weekly_return.shape[0])

#Calculate garch_std sd for each stock
one_year_monthly_return=log_monthly_return.iloc[log_monthly_return.index >= one_year_ago_date]
res_all = one_year_monthly_return - np.mean(one_year_monthly_return)
judge = one_year_monthly_return.shape[0]
for stock in one_year_monthly_return.columns:
    res=res_all[stock]
    h= het_arch(res)
    if judge < 100 or h ==0:
        garch_std = weekly_std

    else:#build Garch Model
        std = []
        if judge <=200:
            p = 0
            q = 6
        else:
            p = 1
            q = 1

        std.append(fit_garch(one_year_monthly_return[stock],p,q))
        garch_std = pd.DataFrame(std,index = one_year_monthly_return.columns)

#Calculate monthly sd for each stock
one_year_monthly_return=log_monthly_return.iloc[log_monthly_return.index >= one_year_ago_date]
monthly_std = np.std(one_year_monthly_return)*np.sqrt(12);

#Calculate semi_monthly sd for each stock
one_year_semi_monthly_return=log_semi_monthly_return.iloc[log_semi_monthly_return.index >= one_year_ago_date]
semi_monthly_std = np.std(one_year_semi_monthly_return)*np.sqrt(24);


In [ ]:
# Calculated MaxDrop
DD_range = 90
total_range = 90
DD_end = yahooDataFrame.iloc[yahooDataFrame.index <= one_year_ago_date]
DD_start = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(total_range)]
max_DD=[-1]*selected_stock_number


if (DD_start.shape[0] != 1 ) and (DD_end.shape[0] != 1):
    DD_start = DD_start.index[-1]
    DD_end = DD_end.index[-1]
    while DD_start > DD_end :
        DD_start = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(total_range)]
        DD_mid = yahooDataFrame.iloc[yahooDataFrame.index <= end-datetime.timedelta(DD_range+total_range)]
        if (DD_start.shape[0] != 1 ) and (DD_mid.shape[0] != 1):
            DD_start = DD_start.index[-1]
            DD_mid = DD_mid.index[-1]
            price = yahooDataFrame.loc[DD_start:DD_mid,:]
            for i in range(len(yahooDataFrame.columns)):
                max_DD[i]=max(max_DD[i],max_drawdown_absolute(stock_return_one_year.iloc[:,i])[0])
            total_range = total_range + 1
        else:
            break

In [ ]:
# Read OPT-data
optdata = pd.read_excel("optdata.xlsx",index_col=0,sheet_name = 1)
P_current = MA_now
P_fwd = optdata.P_fwd.loc[P_current.index]
R_fwd=P_fwd/P_current-1
R_60 = P_current/MA_one_year_ago - 1
R_60_half = P_current/MA_half_year_ago - 1
R_MA_1ago = R_mv_one_year
R_MA_halfago = R_mv_half_year
Rfwd_adjusted = pd.Series(index=R_MA_1ago.index)
for i in R_fwd.index:
    if R_fwd[i] < 0.5*R_60[i]:
        Rfwd_adjusted[i] = R_60_half[i]
    elif R_fwd[i] > 1.3*R_60[i]:
        if 1.3*R_60[i] > 0.7*R_fwd[i]:
            Rfwd_adjusted[i] = 1.3*R_60[i]
        else:
            Rfwd_adjusted[i] = 0.7*R_fwd[i]
    else:
        Rfwd_adjusted[i] = R_fwd[i]
Rfwd_adjusted[Rfwd_adjusted<0]=0

In [ ]:
price_now = today_price.iloc[-1,:]
optimizer = optdata.loc[P_current.index,:]
FFC = pd.DataFrame([R_MA_1ago.values,R_MA_halfago.values,MA_now.values,price_now.values,semi_monthly_std.values,weekly_std.values,MA_one_year_ago.values,MA_half_year_ago.values,max_DD,R_60.values,R_60_half.values,P_fwd.values,Rfwd_adjusted.values],columns = R_MA_1ago.index,index = ["R_MA_1ago","R_MA_halfago","MA_now","price",'std','std_f','MA_1ago','MA_halfago','Fall_MDD','R_60','R_60_half','R_fwd',"Rfwd_adjusted"]).T
preselected_stock = preselected_stock.loc[selected_stock_list,:]
preselected_stock["Rfwd_adjusted"]=Rfwd_adjusted.values
preselected_stock["std_f"]=weekly_std.values
preselected_stock["SR"]=preselected_stock["Rfwd_adjusted"]/preselected_stock["std_f"]
preselected_stock["PR_SR"]=preselected_stock["PR"]*preselected_stock["SR"]**0.5
preselected_stock.to_csv("preselected_Stocks_info.csv")
p=FFC.MA_now
EPS_fwd=optimizer.EPS_fwd
EPS_ttm=optimizer.EPS_ttm
fall=FFC.Fall_MDD
FFC.to_csv("WEDataBank.csv")

PE=p/EPS_fwd
PEG= p/(EPS_fwd*(EPS_fwd/EPS_ttm-1))
PEG[PEG<0]=2000


In [ ]:
preselected_stock

,Swan4,Swan5,Swan6,Swan7,Swan8,Group,avg,PR,Rfwd_adjusted,std_f,SR,PR_SR
Ticker,,,,,,,,,,,,
ABT,0.702188,0.474696,0.311079,1.073150,0.431921,5,0.431202,0.568798,0.148914,0.180855,0.823389,0.516132
CL,0.099788,0.617542,-0.045657,0.784583,0.569564,5,0.431202,0.568798,0.138795,0.162372,0.854795,0.525883
DUK,1.580781,0.771714,-0.322793,0.233701,-0.356250,5,0.431202,0.568798,0.224151,0.119864,1.870037,0.777827
EXC,0.519667,0.794675,0.263389,0.605246,-0.123438,5,0.431202,0.568798,0.527425,0.155517,3.391427,1.047488
JNJ,0.403161,0.739963,0.360346,0.668742,0.555245,5,0.431202,0.568798,0.187548,0.154175,1.216466,0.627347
KO,-0.170562,0.536675,0.325968,0.614677,-0.049526,5,0.431202,0.568798,0.126410,0.166501,0.759213,0.495610
LLY,0.746268,0.920577,0.280671,0.780342,0.061466,5,0.431202,0.568798,0.313108,0.201832,1.551326,0.708450
MCD,1.261978,0.321829,0.130072,0.483323,-0.202955,5,0.431202,0.568798,0.014421,0.137778,0.104666,0.184019
MDLZ,0.033695,0.828315,0.344205,0.089729,0.386481,5,0.431202,0.568798,0.185592,0.152518,1.216852,0.627447


In [ ]:
cov = pd.DataFrame(np.cov(np.exp(one_year_weekly_return)-1,rowvar=False,bias = False)*51,index= one_year_weekly_return.columns, columns= one_year_weekly_return.columns)
E_2 = Rfwd_adjusted/fall
E_3 = Rfwd_adjusted/PEG
E_4 = Rfwd_adjusted
std =  np.std(np.exp(one_year_weekly_return)-1)*np.sqrt(52)


# Optimization

In [ ]:
w0 = 1/selected_stock_number*np.ones(selected_stock_number)
uplim_ini = np.zeros(selected_stock_number)
uplim3 =pd.DataFrame( w1*(1-uplim_ini) + w2 * uplim_ini,index = std.index,columns=["weight"])
for i in uplim3.index:
    if i in ["SPY","IWD"]:
        uplim3.loc[i,:]=0
low_lim =  np.zeros(selected_stock_number)
multi=lambda x: x.T @cov@x

In [ ]:
ef = EfficientFrontier(E_2/100, cov,weight_bounds=(0, w1))
#ef.add_constraint(lambda w: w@E_2== 12)
raw_weights = ef.max_sharpe(0.01)
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
ef.portfolio_performance(verbose=True)


OrderedDict([('ABT', 0.0), ('CL', 0.0), ('DUK', 0.1498), ('EXC', 0.0), ('JNJ', 0.0), ('KO', 0.0), ('LLY', 0.11132), ('MCD', 0.16525), ('MDLZ', 0.0), ('MRK', 0.0), ('NEE', 0.17), ('PEP', 0.00139), ('PFE', 0.06224), ('PG', 0.0), ('SBUX', 0.0), ('SO', 0.17), ('VZ', 0.0), ('WMT', 0.17)])
Expected annual return: 11.0%
Annual volatility: 7.9%
Sharpe Ratio: 1.27


/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:441: UserWarning: The risk_free_rate provided to portfolio_performance is different to the one used by max_sharpe. Using the previous value.
  warnings.warn(


(0.10955401819587701, 0.07861900280456699, 1.2662844178188166)

In [ ]:
weight1=np.array(pd.DataFrame(cleaned_weights.values(), index = list(cleaned_weights),columns = ["weight"]))[:,0]

In [ ]:
sd1 = np.sqrt(weight1@cov@weight1.T)
R1 = weight1.T@E_2/100
sharpe_ratio1 = (R1-0.01)/sd1
print("new_sharpe:",sharpe_ratio1)

new_sharpe: 1.2662844177023937


weight2 =   pd.read_excel("old_weight.xlsx",index_col=0)
weight2=np.array(weight2.loc[list(cleaned_weights),:])[:,0]

sd2 = np.sqrt(weight2@cov@weight2.T)
R2 = weight2.T@E_2/100
sharpe_ratio2 = (R2-0.01)/sd2
print("old_sharpe:",sharpe_ratio2)

In [ ]:
weight=pd.DataFrame(cleaned_weights.values(), index = list(cleaned_weights),columns = ["weight"])
weight = weight[weight.weight >=0.03]
weight.weight = weight.weight/sum(weight.weight)
weight.to_csv("weights.csv")

In [ ]:
weight

,weight
DUK,0.150009
LLY,0.111475
MCD,0.165480
NEE,0.170237
PFE,0.062327
SO,0.170237
WMT,0.170237


See pyportfolioopt.readthedocs.io/en/latest/MeanVariance.html for more reference


In [ ]:
if stock_type == "S&P100":
    OEX= web.DataReader("^OEX", 'yahoo', start=start_swan_date, end=end_swan_date)["Adj Close"]
elif stock_type == "S&P500":
    OEX= web.DataReader("SPY", 'yahoo', start=start_swan_date, end=end_swan_date)["Adj Close"]
stock_price=web.DataReader(weight.index, 'yahoo', start=start_swan_date, end=end_swan_date)["Adj Close"]

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
start=stock_price.loc["2020-02-19"]
end = stock_price.loc["2020-03-23"]
stock_drop=abs(end/start-1)
market_drop = abs(OEX.loc["2020-03-23"]/OEX.loc["2020-02-19"]-1)

In [ ]:
beta = stock_drop/market_drop
1-beta.T@weight

weight    0.309557
dtype: float64

In [ ]:
stock_drop

Symbols
MRK     0.183554
SO      0.369917
MO      0.304829
LLY     0.154654
COST    0.116061
NEE     0.335170
KO      0.366080
WMT     0.024595
dtype: float64

In [ ]:
print(stock_price.head(1))
weight


Symbols           MRK         SO         MO         LLY        COST       NEE  \
Date                                                                            
2020-02-18  71.769905  62.606308  36.321125  135.411057  307.424561  66.67382   

Symbols            KO         WMT  
Date                               
2020-02-18  54.581978  114.461243  


,weight
MRK,0.059568
SO,0.170337
MO,0.049819
LLY,0.137562
COST,0.170337
NEE,0.170337
KO,0.071702
WMT,0.170337


In [ ]:
a=stock_price@weight
b=1-a.loc["2020-03-23"]/a.loc["2020-02-19"]
1-b/market_drop

weight    0.484297
dtype: float64

In [ ]:
injected_result = pd.DataFrame(index = stock_price.index)
S1 = np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T


,0
Date,
2020-02-18,10000.000000
2020-02-19,9910.394486
2020-02-20,9978.828339
2020-02-21,9971.371760
2020-02-24,9757.785872
...,...
2021-03-17,10762.485511
2021-03-18,10623.478069
2021-03-19,10664.095915


In [ ]:
portfolio_value = np.sum(np.array(weight.T) * stock_price,axis=1)
S1 = np.array(10000*weight.T)/np.array(stock_price.iloc[0,:]).T
injected_result["portfolio_value"]=stock_price@S1.T
S2 = 10000/float(OEX[0])
injected_result["oex"]=S2*OEX
injected_result.to_csv("Injected_result.csv")

In [ ]:
allocation=pd.read_excel('allocation weight.xlsx',index_col='stock selected')
dfMVOstock=deepcopy(weight)
dfMVOstock['PR_SR']=''
dfMVOstock['SR']=''
for i in range(len(weight)):
    dfMVOstock.iloc[i,1]=preselected_stock.loc[weight.index[i],'PR_SR']
    dfMVOstock.iloc[i,2]=preselected_stock.loc[weight.index[i],'SR']
dfMVOstock_sort=dfMVOstock.sort_values(by='PR_SR',ascending=False)
postallocationweight=post_allocation(dfMVOstock_sort)
postallocationweight


In [ ]:
#draw type E graph for postallocation
MVOweight=postallocationweight['weight']
equalweight=postallocationweight['equal']
proportionweight=postallocationweight['adjusted proportion']
directweight=postallocationweight['direct']
output(MVOweight).to_csv('no_post_allocation_result.csv')
output(equalweight).to_csv('equal_weight_result.csv')v
output(proportionweight).to_csv('proportional_weight_result.csv')
output(directweight).to_csv('direct_weight_result.csv')

#